# Automate the boring stuff with tm1py
## Christoph Hein – DB Systel GmbH

---
## Getting started
Prerequisites
- Any Planning Analytics Version
- Open HTTPPort (firewall, etc. )
- Python 3 and tm1py (maybe other awesome packages)



In [ ]:
from TM1py.Services import TM1Service

# credentials
tm1_credentials = {
    "address":"localhost", 
    "port": "5001",
    "user": "admin", 
    "password": "apple", 
    "ssl": "True"
}



# other useful modules
import pandas as pd
from matplotlib import pyplot
import numpy as np
import networkx as nx
from netwulf import visualize
import re
from IPython.display import Markdown, display

---
## Configuration
### Get product version

In [ ]:
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user']
, password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    tm1Version = tm1.server.get_product_version()
    print('Current TM1 version is ' + tm1Version)

### Change configuration

In [ ]:
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user']
, password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    staticConfiguration = tm1.server.get_static_configuration()
    print('NumberOfThreadsToUse is ' + str(staticConfiguration['Performance']['MTQ']['NumberOfThreadsToUse']))

In [ ]:
changedConfiguration = {'Performance':{'MTQ': {'NumberOfThreadsToUse':4}}}
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user']
, password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    tm1.server.update_static_configuration(changedConfiguration)
    staticConfiguration = tm1.server.get_static_configuration()
    print('NumberOfThreadsToUse is ' + str(staticConfiguration['Performance']['MTQ']['NumberOfThreadsToUse']))


### Monitor usage

In [ ]:
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user']
, password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    activeUsers = tm1.monitoring.get_active_users()

In [ ]:
# do something with the data
df = pd.DataFrame(columns=['name', 'friendlyname','type'])
for x in activeUsers:
    df = df.append({'name': x._name, 'friendlyname': x._friendly_name, 'type': x.user_type  }, ignore_index=True)
print(df)

---
## Message log
### Get startup time

In [ ]:
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user']
, password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    messagelog = tm1.server.get_message_log_entries(reverse=False )

In [ ]:
df = pd.DataFrame(columns=['timestamp', 'duration'])
for x in messagelog:
    if x['Message'].find('TM1 Server is ready, elapsed time ') > -1:
        f_StartupTime  = float(x['Message'][34:x['Message'].find(' seconds')])
        df = df.append({'timestamp': x['TimeStamp'][:-1], 'duration': f_StartupTime}, ignore_index=True)
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%dT%H:%M:%S.%f')
df = df.set_index('timestamp')
print(df)

In [ ]:
#plot it
df['duration'].plot(linewidth=0.5)
pyplot.show()

### Calculate feeder load time

In [ ]:
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user']
, password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    messagelog = tm1.server.get_message_log_entries(reverse=False )
    cubeContent = tm1.cubes.get_all()

In [ ]:
# get a list of cubes with feeders
listOfCubesWithFeeders = []
for x in cubeContent:
    if x.has_rules == True:
        if x.rules.has_feeders == True:
            listOfCubesWithFeeders.append(x._name)
print(listOfCubesWithFeeders)

In [ ]:
df = pd.DataFrame(columns=['timestamp_start', 'timestamp_end','cube','new' , 'duplicate' , 'ignored' ])
# parse message log,
for x in messagelog:

    # compare startup time server 
    s_Date = x['TimeStamp'][0:10]
    s_DateTime = x['TimeStamp'][:-1]
    s_LogType = x['Logger']
    s_Message = x['Message']

    # normally processed feeders
    if x['Message'].find('Feeders created in cube ') > -1:
        s_CubeName = x['Message'][x['Message'].find('cube "')+6:x['Message'].find('": new')]
        n_new = int(x['Message'][x['Message'].find(': new ')+6:x['Message'].find(', duplicate')])
        n_duplicate = int(x['Message'][x['Message'].find(', duplicate ')+12:x['Message'].find(', ignored')])
        n_ignored = int(x['Message'][x['Message'].find(', ignored ')+10:])
        b_Add2DF = True
        
    # Persistent Feeders
    elif s_Message.find('Feeders already loaded for cube ') > -1:
        s_CubeName = s_Message[s_Message.find('cube "')+6:s_Message.find('": new')]
        n_new = 0
        n_duplicate = 0
        n_ignored = 0
        b_Add2DF = True

        if s_CubeName in listOfCubesWithFeeders & b_Add2DF == True:
            df = df.append({'timestamp_start': s_PrevTimestamp, 'timestamp_end': s_DateTime, 'cube': s_CubeName,'new':n_new , 'duplicate':n_duplicate , 'ignored':n_ignored  }, ignore_index=True)

    s_PrevTimestamp = s_DateTime

df['timestamp_start'] = pd.to_datetime(df['timestamp_start'], format='%Y-%m-%dT%H:%M:%S.%f')
df['timestamp_end'] = pd.to_datetime(df['timestamp_end'], format='%Y-%m-%dT%H:%M:%S.%f')

df['duration'] = df['timestamp_end'] - df['timestamp_start']
df['duration'] = (df['duration']/np.timedelta64(1,'s'))

print(df)


---
## Processes
### Generate documentation

In [ ]:
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user']
, password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    processes =  tm1.processes.get_all()

In [ ]:
for process in processes:
        if process.parameters :
            display(Markdown('**' + process.name + '**'))
            listParameters = []
            for parameter in process.parameters:
                listParameters.append(parameter['Name'] + ' (*' + parameter['Type'] + '*), ')
            mystring = ''.join(listParameters)
            display(Markdown(mystring))

---
## User management
#### Get user information from instance

In [ ]:
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user']
, password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    users =  tm1.security.get_all_users()

In [ ]:
df = pd.DataFrame(columns=['server','user','type' , 'status' ])
for user in users:
    df = df.append({'server': 'server01','user': user._name.lower().replace(" ", ""),'type':str(user._user_type) , 'status':str(user._enabled ) }, ignore_index=True)
print(df)

#### Add another instance

In [ ]:
with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user']
, password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    users =  tm1.security.get_all_users()

In [ ]:
for user in users:
    df = df.append({'server': 'server02','user': user._name.lower().replace(" ", ""),'type':str(user._user_type) , 'status':str(user._enabled ) }, ignore_index=True)
print(df)

#### edge cases
##### read only users
The read only flag is currently not exposed via the rest api. 
My personal opinion is that ibm SHOULD expose it. But for now we need to use a workaround. 

In [ ]:

with TM1Service(address=tm1_credentials['address'], port=tm1_credentials['port'], user=tm1_credentials['user']
, password=tm1_credentials['password'],ssl=tm1_credentials['ssl'] ) as tm1: 
    cubeContent =  tm1.cells.execute_mdx_rows_and_values('SELECT {[}ClientProperties].[ReadOnlyUser]} ON COLUMNS, {[}Clients].MEMBERS} ON ROWS FROM [}ClientProperties]')

In [ ]:
df = pd.DataFrame(columns=['ipadress', 'port','user','type' , 'status' ])
for user in cubeContent:
    username = str(user[0]).replace('[}Clients].[}Clients].[','').replace(']','')
    if str(cubeContent[user][0]) == 'None':
        s_userright = 'WRITE'
    else:
        s_userright = 'READ'
    df = df.append({'ipadress': tm1_credentials['address'], 'port': tm1_credentials['port'],'user': username,'type':s_userright , 'status':'NA' }, ignore_index=True)
print(df)



##### CAM user
CAM users in tm1 have no proper user id in the client dimension. Therefore the display name must be used. 
Please notice that that might not always bring the intended results. 

In [ ]:
# with CAM use friendly name instead to avoid CAM user id and maybe do some string operations
listOfCAMUsers = ['MyDomain\\FirstUser', 'Mydomain\\SecondUser', 'mydomain//ThirdUser']
for user in listOfCAMUsers:
    user_Name = user.lower().replace('mydomain', '').replace('\\', '').replace('/', '')
    print(user_Name)